#Installation

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

!conda update
!conda install -c bioconda cobra
!pip install cameo[all]



⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:16
🔁 Restarting kernel...

CondaValueError: no package names supplied
# Example: conda update -n myenv scipy


Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | 

#Cameo

Import models from the internet
In the quick start chapter we demonstrated how to use load_model() to import a model by ID. But where did the model come from? Cameo has currently access to two model repositories on the internet, http://bigg.ucsd.edu and http://darwin.di.uminho.pt/models.

##import

In [ ]:
from cameo import models
models.index_models_bigg()      #check models in bigg

In [ ]:
#check models in minho
from cameo import models
models.index_models_minho()


,id,name,doi,author,year,formats,organism,taxonomy,validated
0,1,iJR904,10.1186/gb-2003-4-9-r54,Reed,2003,[sbml],Escherichia coli str. K12 substr. MG1655,Bacteria; Proteobacteria; Gammaproteobacteria;...,True
1,2,iAF1260,10.1038/msb4100155,Feist,2007,[sbml],Escherichia coli str. K12 substr. MG1655,Bacteria; Proteobacteria; Gammaproteobacteria;...,True
2,3,iMM904,10.1186/1752-0509-3-37,Mo,2007,[sbml],Saccharomyces cerevisiae,Eukaryota; Opisthokonta; Fungi; Dikarya; Ascom...,True
3,4,iJP815,10.1371/journal.pcbi.1000210,Puchalka,2008,[sbml],Pseudomonas putida str. KT2440,Bacteria; Proteobacteria; Gammaproteobacteria;...,True
4,5,iMO1056,10.1128/JB.01583-07,Oberhardt,2008,[excel],Pseudomonas aeruginosa str. PAO1,Bacteria; Proteobacteria; Gammaproteobacteria;...,False
...,...,...,...,...,...,...,...,...,...
145,150,iLB1027_lipid,10.1371/journal.pone.0155038,Jennifer Levering,2016,[sbml],Phaeodactylum tricornutum,Eukaryota; Stramenopiles; Bacillariophyta; Bac...,True
146,151,iLB1027,10.1371/journal.pone.0155038,Jennifer Levering,2016,[sbml],Phaeodactylum tricornutum,Eukaryota; Stramenopiles; Bacillariophyta; Bac...,True
147,152,iMT1174,10.1186/s12918-015-0190-y,Mohammad Tajparast,2015,[excel],Rhodococcus jostii RHA1,Bacteria; Terrabacteria group; Actinobacteria;...,False
148,153,iBP722,doi:10.1186/s12918-018-0585-7,B Pereira,2018,[],Actinobacillus succinogenes 130Z,Bacteria; Proteobacteria; Gammaproteobacteria;...,True


In [ ]:
#downdload models from minho or BIGG
#models.minho."ID"

models.minho.iMM904
models.bigg.iJN746


A saída de streaming foi truncada nas últimas 5000 linhas.
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_leu_L_c "M_L_Leucine_C6H13NO2">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_leu_L_e "M_L_Leucine_C6H13NO2">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_leu_L_m "M_L_Leucine_C6H13NO2">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_leu_L_v "M_L_Leucine_C6H13NO2">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_leutrna_c "M_L_Leucyl_tRNA_Leu__C6H12NOR">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_leutrna_m "M_L_Leucyl_tRNA_Leu__C6H12NOR">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_lgt_S_c "M__R__S_Lactoylglutathione_C13H20N3O8S">
Use of the species charge attribute is discouraged, use fbc:charge i

Name,iJN746
Memory address,0x07f81de3b33d0
Number of metabolites,907
Number of reactions,1054
Number of groups,0
Objective expression,1.0*BIOMASS_KT_TEMP - 1.0*BIOMASS_KT_TEMP_reverse_d18f7
Compartments,"extracellular space, cytosol, periplasm"


In [ ]:
#aqui estou baixando o modelo iRC1080 do BIGG, que é o modelo da chlamydomonas reinhardtii
model = models.bigg.iRC1080


Models in the Minho database have been manually verified. The subset of models shown bellow can be used to run simulations as described in the publications.

models.minho.validated.VvuMBEL943 # use TAB completion to see the other models

In [ ]:
models.bigg.validated.iRC1080

##Simulate models FBA
cameo uses the model data structures defined by cobrapy, our favorite COnstraints-Based Reconstruction and Analysis tool for Python. cameo is thus 100% compatible with cobrapy. For efficiency reasons, however, cameo implements its own simulation methods that take advantage of a more advanced solver interface.

In [ ]:
from cameo import load_model


model_id = 'iRC1080'            #iRC1080 is from clamydomonas r.
model = load_model(model_id)    #import model by name


from cameo import fba
%time fba_result = fba(model, objective='BIOMASS_Chlamy_hetero')  #In cameo, flux balance analysis can be performed with the function fba.



CPU times: user 403 ms, sys: 3.12 ms, total: 406 ms
Wall time: 403 ms


Basically, fba calls model.optimize() and wraps the optimization solution in a FluxDistributionResult object. The maximum objective values (corresponding to a maximum growth rate) can be obtained from result.objective_value.

In [ ]:
fba_result.data_frame

,flux
EX_hxan_e,0.0
EX_alltn_e,0.0
EX_orn_e,0.0
EX_gln__L_e,0.0
EX_urate_e,0.0
...,...
PYDXOR,0.0
PYDXPP,0.0
ACHMSSELCYSL,0.0
PYDXS,0.0


In [ ]:
#Flux distributions can be visualized using escher :
#!pip install escher

#HOW TO CREATE A NEW ESCHER MAP FOR MY SAMPLE?

fba_result.display_on_map("iJO1366.Central metabolism")

###Parsimonious Flux Balance Analysis

Parsimonious flux balance analysis (Lewis et al., 2010), a variant of FBA, performs FBA in in a first step to determine the maximum objective value , fixes it in form of an additional model constraint (), and then minimizes the  norm of . The assumption behind pFBA is that cells try to minimize flux magnitude as well in order to keep protein costs low.

In [ ]:
from cameo import pfba
%time pfba_result = pfba(model, objective='BIOMASS_Chlamy_hetero')

CPU times: user 233 ms, sys: 4.11 ms, total: 237 ms
Wall time: 240 ms


In [ ]:
pfba_result.objective_value

#… which is smaller than flux vector of the original FBA solution.  4184.9 is smaller than 4790

7391.261282559429

In [ ]:
abs(fba_result.data_frame.flux).sum()

8039.558883988466

###Setp 2: Simulate knockouts phenotypes
Although PFBA and FBA can be used to simulate the effect of knockouts, other methods have been proven more valuable for that task: MOMA and ROOM. In cameo we implemented a linear version of MOMA.

In [ ]:
#mulating knockouts:

#Manipulate the bounds of the reaction (or use the shorthand method knock_out)

model.reactions.GAPDH_nadp_hi     # I choose the GAPDH_nadp_hi from list model.reactions

Reaction identifier,GAPDH_nadp_hi
Name,Glyceraldehyde-3-phosphate dehydrogenase (NADP+) (phosphorylating)
Memory address,0x07f120878a750
Stoichiometry,13dpg_h + h_h + nadph_h --> g3p_h + nadp_h + pi_h 3-Phospho-D-glyceroyl phosphate + H+ + Nicotinamide adenine dinucleotide phosphate - reduced --> Glyceraldehyde 3-phosphate + Nicotinamide adenine dinucleotide phosphate + Phosphate
GPR,CRv4_Au5_s1_g1191_t1
Lower bound,0.0
Upper bound,1000.0


In [ ]:
#knockout of GAPDH_nadp_hi

model.reactions.GAPDH_nadp_hi.knock_out()
model.reactions.GAPDH_nadp_hi

Reaction identifier,GAPDH_nadp_hi
Name,Glyceraldehyde-3-phosphate dehydrogenase (NADP+) (phosphorylating)
Memory address,0x07f120878a750
Stoichiometry,13dpg_h + h_h + nadph_h --> g3p_h + nadp_h + pi_h 3-Phospho-D-glyceroyl phosphate + H+ + Nicotinamide adenine dinucleotide phosphate - reduced --> Glyceraldehyde 3-phosphate + Nicotinamide adenine dinucleotide phosphate + Phosphate
GPR,CRv4_Au5_s1_g1191_t1
Lower bound,0
Upper bound,0


In [ ]:
#brincando de deletar, vou deletar PRISM_solar_litho

model.reactions.PRISM_solar_litho.knock_out()
model.reactions.PRISM_solar_litho

Reaction identifier,PRISM_solar_litho
Name,Spectral decomposition of solar light measured from Earth's surface
Memory address,0x07f12087f4a10
Stoichiometry,"photonVis_e --> 4.26984325300204e-05 photon298_c + 0.088253056987783 photon437_u + 0.187390761056649 photon438_u + 0.114958292879089 photon450_h + 0.195814806722332 photon490_s + 0.18566429641716... Photon (380 to 750 nm, visible spectrum) --> 4.26984325300204e-05 Photon (281 to 306 nm, UVB) + 0.088253056987783 Photon (406 to 454 nm, indigo/blue) + 0.187390761056649 Photon (378 to 482 nm,..."
GPR,
Lower bound,0
Upper bound,0


In [ ]:
%time fba_knockout_result = fba(model)
fba_knockout_result[model.reactions.BIOMASS_Chlamy_hetero]

CPU times: user 118 ms, sys: 940 µs, total: 119 ms
Wall time: 121 ms


0.0

In [ ]:
%time pfba_knockout_result = pfba(model)
pfba_knockout_result[model.reactions.BIOMASS_Chlamy_hetero]

CPU times: user 826 ms, sys: 6.01 ms, total: 832 ms
Wall time: 834 ms


0.0

In [ ]:
#Parsimonious FBA references seem to produce better results using this methods

from cameo.flux_analysis.simulation import room, lmoma

%time lmoma_result = lmoma(model, reference=pfba_result.fluxes)
lmoma_result[model.reactions.BIOMASS_Chlamy_hetero]


#what is exactly this number?
#AWNSER= BIOMASS_Chlamy_hetero

CPU times: user 15.2 s, sys: 176 ms, total: 15.4 s
Wall time: 16.6 s


6.3733278331168846

###Analyzing models
cameo uses the model data structures defined by cobrapy, our favorite COnstraints-Based Reconstruction and Analysis tool for Python. cameo is thus 100% compatible with cobrapy. For efficiency reasons though cameo implements its own analysis methods that take advantage of a more advanced solver interface.

In [ ]:
from cameo import models
from cameo.visualization.plotting.with_plotly import PlotlyPlotter

model = models.bigg.iRC1080     #IRC1080 is from clamydomonas... We can use models.bigg.'id
plotter = PlotlyPlotter()

Flux Variability Analysis
Flux variability analysis (FVA) enables the computation of lower and upper bounds of reaction fluxes.

In [ ]:
from cameo import flux_variability_analysis

fva_result = flux_variability_analysis(model)
fva_result.data_frame

,lower_bound,upper_bound
EX_hxan_e,0.0,0.000000
EX_alltn_e,0.0,0.000000
EX_orn_e,0.0,0.000000
EX_gln__L_e,0.0,0.000000
EX_urate_e,0.0,0.000000
...,...,...
PYDXOR,0.0,455.823274
PYDXPP,0.0,621.925866
ACHMSSELCYSL,0.0,0.000000
PYDXS,0.0,0.000000


In [ ]:
fva_result.plot(plotter, index=fva_result.data_frame.index[:25])

In [ ]:
#One very useful application of FVA is determining if alternative optimal solution exist.

fva_result2 = flux_variability_analysis(model,fraction_of_optimum=0.5)
fva_result2.data_frame

,lower_bound,upper_bound
EX_hxan_e,0.000000e+00,0.000000e+00
EX_alltn_e,0.000000e+00,0.000000e+00
EX_orn_e,0.000000e+00,0.000000e+00
EX_gln__L_e,0.000000e+00,0.000000e+00
EX_urate_e,0.000000e+00,0.000000e+00
...,...,...
PYDXOR,0.000000e+00,4.558233e+02
PYDXPP,0.000000e+00,6.219259e+02
ACHMSSELCYSL,0.000000e+00,0.000000e+00
PYDXS,0.000000e+00,0.000000e+00


In [ ]:
fva_result2.plot(plotter)

##Phenotypic Phase Plane
The phenotypic phase plane is a modeling technique was developed to do a theoretical accessement of what cell can or cannot do in terms of the stoichiometric constraints [Edawards et al. 2001].

The phenotipic phase plane between growth and a product of interest yields the production envelope: a representation between the trade of between production of the desired product and growth.


In [ ]:
model.objective = 'BIOMASS_Chlamy_hetero'
print (model.objective)
model.reactions.BIOMASS_Chlamy_hetero

Maximize
1.0*BIOMASS_Chlamy_hetero - 1.0*BIOMASS_Chlamy_hetero_reverse_b1c3b


Reaction identifier,BIOMASS_Chlamy_hetero
Name,Biomass (heterotrophic)
Memory address,0x07f81d9f4fd10
Stoichiometry,0.0370621525868795 ac_c + 7.9745622797189e-08 acaro_h + 0.309071177207791 ala__L_c + 1.59491245594378e-08 anxan_u + 0.524141412987151 arab__L_c + 0.0949756387513779 arg__L_c + 0.0651648675739026... 0.0370621525868795 Acetate + 7.9745622797189e-08 Alpha-Carotene + 0.309071177207791 L-Alanine + 1.59491245594378e-08 Antheraxanthin + 0.524141412987151 L-Arabinose + 0.0949756387513779 L-Arginine +...
GPR,
Lower bound,0.0
Upper bound,1000.0


In [ ]:
model.reactions.EX_etoh_e

Reaction identifier,EX_etoh_e
Name,Ethanol exchange
Memory address,0x07f81da30ab10
Stoichiometry,etoh_e --> Ethanol -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [ ]:
from cameo import phenotypic_phase_plane
model.reactions.EX_etoh_e.lower_bound = -1000
result = phenotypic_phase_plane(model,
                                variables=[model.reactions.BIOMASS_Chlamy_hetero],
                                objective=model.reactions.EX_etoh_e,
                                points=10)

In [ ]:
result.plot(plotter)

The production envelope allows is a quick way to inspect the limitations of the system to design and how the production relates for with growth. In the previous example, succinate prudction is completly decoupled from growth and by decreasing the growth rate it is theoretically possible to produce up to 15 times more succinate.

In [ ]:
result = phenotypic_phase_plane (model,
                                 variables=[model.reactions.BIOMASS_Chlamy_hetero],
                                 objective=model.reactions.EX_etoh_e)
result.plot(plotter)

In [ ]:
result.data_frame  #The computed data can be inspected in the format of a pandas data frame by calling result.data_frame

,BIOMASS_Chlamy_hetero,objective_lower_bound,objective_upper_bound,c_yield_lower_bound,c_yield_upper_bound,mass_yield_lower_bound,mass_yield_upper_bound
0,0.000000,0.0,378.039556,0.0,37.803956,0.0,29.496116
1,0.335438,0.0,372.360559,0.0,37.236056,0.0,29.053019
2,0.670877,0.0,366.681563,0.0,36.668156,0.0,28.609921
3,1.006315,0.0,361.002566,0.0,36.100257,0.0,28.166824
4,1.341753,0.0,355.323570,0.0,35.532357,0.0,27.723726
5,1.677192,0.0,349.644574,0.0,34.964457,0.0,27.280629
6,2.012630,0.0,343.965577,0.0,34.396558,0.0,26.837532
7,2.348068,0.0,338.286581,0.0,33.828658,0.0,26.394434
8,2.683506,0.0,332.607584,0.0,33.260758,0.0,25.951337
9,3.018945,0.0,326.928588,0.0,32.692859,0.0,25.508239


In [ ]:
#Flux Balance Impact Degree


from cameo.flux_analysis.analysis import flux_balance_impact_degree
model.reactions.EX_etoh_e.lower_bound = -1000
%time fbid = flux_balance_impact_degree(model, ["EX_etoh_e"])


CPU times: user 46.3 s, sys: 130 ms, total: 46.4 s
Wall time: 46.4 s


In [ ]:
fbid

,perturbed
EX_etoh_e,False
EX_h2_e,False
EX_succ_e,True
EX_h_e,False
EX_h2o_e,False
...,...
ACP1_FMN,False
PYDXNK,False
PYDXNO,False
PYDXOR,False


###predict gene knockout strategies

In cameo we have two ways of predicting gene knockout targets: using evolutionary algorithms (OptGene) or linear programming (OptKnock)

If you’re running this notebook on try.cameo.bio, things might run very slow due to our inability to provide access to the proprietary CPLEX solver on a public webserver. Furthermore, Jupyter kernels might crash and restart due to memory limitations on the server.

In [ ]:
from cameo import models
from cameo.visualization.plotting.with_plotly import PlotlyPlotter

In [ ]:
plotter = PlotlyPlotter()

In [ ]:
#I still use hthe example from biomass and ethanol production

wt_solution = model.optimize()
growth = wt_solution.fluxes["BIOMASS_Chlamy_hetero"]
ethanol_production = wt_solution.fluxes["EX_etoh_e"]

In [ ]:
from cameo import phenotypic_phase_plane
p = phenotypic_phase_plane(model, variables=['BIOMASS_Chlamy_hetero'], objective='EX_etoh_e')
p.plot(plotter, points=[(growth, ethanol_production)])

###OptGene
OptGene is an approach to search for gene or reaction knockouts that relies on evolutionary algorithms[1]. The following image from the authors summarizes OptGene workflow.

At every iteration, we keep the best 50 individuals found overall so we can generate a library of targets.

In [ ]:
from cameo.strain_design import OptGene
optgene = OptGene(model)
result = optgene.run(target=model.reactions.EX_etoh_e,
                     biomass=model.reactions.BIOMASS_Chlamy_hetero,
                     substrate=model.metabolites.glu__L_h,
                     max_evaluations=100,
                     plot=False)


Starting optimization at Sun, 26 Dec 2021 23:00:54


HBox()

Finished after 00:00:52


In [ ]:
result

,reactions,genes,size,fva_min,fva_max,target_flux,biomass_flux,yield,fitness


In [ ]:
result.display_on_map(0, "iJO1366.Central metabolism")

In [ ]:
from cameo.strain_design import OptKnock
optknock = OptKnock(model, fraction_of_optimum=0.1)

/usr/local/lib/python3.7/site-packages/cameo/strain_design/deterministic/linear_programming.py:119: UserWarning:

You are trying to run OptKnock with glpk_interface. This might not end well.



In [ ]:
result = optknock.run(max_knockouts=1, target="EX_etoh_e", biomass="BIOMASS_Chlamy_hetero")

In [ ]:
result.plot(plotter, 0)

HBox()

In [ ]:
result

,reactions,size,EX_etoh_e,biomass,fva_min,fva_max
0,{TRE6PP},1,378.039556,0.0,0.0,378.039556
1,{UG6PGT},1,378.039556,0.0,0.0,378.039556


In [ ]:
result.display_on_map(0, "iJO1366.Central metabolism")

In [ ]:
model.reactions.TRE6PP

Reaction identifier,TRE6PP
Name,Trehalose-phosphatase
Memory address,0x07f81d839be50
Stoichiometry,"h2o_c + tre6p_c --> pi_c + tre_c H2O H2O + Alpha,alpha'-Trehalose 6-phosphate --> Phosphate + Trehalose"
GPR,CRv4_Au5_s12_g3028_t1 or CRv4_Au5_s13_g5046_t1
Lower bound,0.0
Upper bound,1000.0


In [ ]:
model.reactions.UG6PGT

Reaction identifier,UG6PGT
Name,UDPglucose:D-glucose-6-phosphate 1-alpha-D-glucosyltransferase
Memory address,0x07f81dcd4fa50
Stoichiometry,"g6p_A_c + udpg_c --> h_c + tre6p_c + udp_c Alpha-D-glucose 6 phosphate + UDPglucose --> H+ + Alpha,alpha'-Trehalose 6-phosphate + UDP C9H11N2O12P2"
GPR,CRv4_Au5_s13_g5046_t1 or CRv4_Au5_s16_g6776_t1
Lower bound,0.0
Upper bound,1000.0
